# Simple MLOps Pipeline with API V2

In [51]:
import os
import cmlapi
from cmlapi.utils import Cursor
import string
import random
import json
import yaml
import datetime

In [52]:
# Install cmlapi package
try:
    import cmlapi
except ModuleNotFoundError:
    import os
    cluster = os.getenv("CDSW_API_URL")[:-1]+"2"
    !pip3 install {cluster}/python.tar.gz
    import cmlapi

from cmlapi.utils import Cursor
import string
import random
import json

try:
    client = cmlapi.default_client()
except ValueError:
    print("Could not create a client. If this code is not being run in a CML session, please include the keyword arguments \"url\" and \"cml_api_key\".")

In [53]:
session_id = "".join([random.choice(string.ascii_lowercase) for _ in range(6)])
session_id

'jcxnli'

In [54]:
run_time_suffix = datetime.datetime.now()
run_time_suffix = run_time_suffix.strftime("%d%m%Y%H%M%S")

In [55]:
project_id = os.environ["CDSW_PROJECT_ID"]

In [57]:
model_name = "demo-model-" + session_id

In [58]:
# Create the YAML file for the model lineage
# This could be done inside a method
yaml_text = \
""""{}":
  hive_table_qualified_names:                # this is a predefined key to link to training data
  
    - "default.marketing_campaign_table@cm"  # the qualifiedName of the hive_table object representing 
    
    - "default.bank_info_table@cm"
  metadata:                                  # this is a predefined key for additional metadata
  
    additional_timestamp: {}                 # suggested use case: query used to extract training data
""".format(model_name, run_time_suffix)

with open('lineage.yml', 'w') as lineage: lineage.write(yaml_text)

In [59]:
modelReq = cmlapi.CreateModelRequest(
    name = "demo-model-" + session_id,
    description = "model created for demo",
    project_id = project_id,
    disable_authentication = True
)

model = client.create_model(modelReq, project_id)

In [60]:
model_build_request = cmlapi.CreateModelBuildRequest(
    project_id = project_id,
    model_id = model.id,
    comment = "test comment",
    file_path = "models/model_endpoint.py",
    function_name = "predict",
    kernel = "python3",
    runtime_identifier = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-standard:2022.04.1-b6"
    #runtime_addon_identifiers = "spark311-13-hf1"
)

modelBuild = client.create_model_build(
    model_build_request, project_id, model.id
)

In [61]:
model_deployment = cmlapi.CreateModelDeploymentRequest(
        project_id = project_id, 
        model_id = model.id, 
        build_id = modelBuild.id, 
        cpu = 1.00,
        memory = 2.00
    )

model_deployment_response = client.create_model_deployment(
        model_deployment, 
        project_id = project_id, 
        model_id = model.id, 
        build_id = modelBuild.id
    )

In [40]:
## CREATE MODEL PROGRAMMATICALLY ##

# Create the YAML file for Model Lineage

#name = "demo-model-" + session_id

#dict_yaml = [{name : {"hive_table_qualified_names": ["default.marketing_campaign_table", "default.bank_info_table"], "metadata":{"custom metadata key" : '"custom metadata val"'}}}]

#with open(r'lineage.yml', 'w') as file:
#    documents = yaml.dump(dict_yaml, file)